<a href="https://colab.research.google.com/github/pratik9153/Spark-ML/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('Adult Income').getOrCreate()

In [9]:
df = spark.read.csv("Adult.data.csv")

In [10]:
df.show()

+---+----------------+------+------------+-------------+--------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
|_c0|             _c1|   _c2|         _c3|          _c4|           _c5|              _c6|           _c7|  _c8|   _c9|        _c10|        _c11|          _c12|          _c13|  _c14|
+---+----------------+------+------------+-------------+--------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|   education|education.num|marital.status|       occupation|  relationship| race|   sex|capital.gain|capital.loss|hours.per.week|native.country|income|
| 90|               ?| 77053|     HS-grad|            9|       Widowed|                ?| Not-in-family|White|Female|           0|        4356|            40| United-States| <=50K|
| 82|         Private|132870|     HS-grad|            9|       Widowed|  Exec-managerial| Not-i

In [11]:
labels = ['age', 'workclass', 'fnlwgt', 'education', 'numbers', 'marital', 'occupation', 'relation', 'race', 'gender', 'gain', 'loss', 'hourlypay', 'country', 'income']

In [13]:
df = df.toDF(*labels)

In [14]:
df.show(5)

+---+---------+------+------------+-------------+--------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|      numbers|       marital|       occupation|     relation| race|gender|        gain|        loss|     hourlypay|       country|income|
+---+---------+------+------------+-------------+--------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|education.num|marital.status|       occupation| relationship| race|   sex|capital.gain|capital.loss|hours.per.week|native.country|income|
| 90|        ?| 77053|     HS-grad|            9|       Widowed|                ?|Not-in-family|White|Female|           0|        4356|            40| United-States| <=50K|
| 82|  Private|132870|     HS-grad|            9|       Widowed|  Exec-managerial|Not-in-family|White|Female|           0|        4356|

In [15]:
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- numbers: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relation: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- gain: string (nullable = true)
 |-- loss: string (nullable = true)
 |-- hourlypay: string (nullable = true)
 |-- country: string (nullable = true)
 |-- income: string (nullable = true)



In [16]:
from pyspark.sql.functions import col
df_1 = df.withColumn('age',col('age').cast('integer'))

In [17]:
df_1.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- numbers: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relation: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- gain: string (nullable = true)
 |-- loss: string (nullable = true)
 |-- hourlypay: string (nullable = true)
 |-- country: string (nullable = true)
 |-- income: string (nullable = true)



In [20]:
for i in ['fnlwgt' , 'numbers', 'gain', 'loss', 'hourlypay']:
  df_1 = df_1.withColumn(i,col(i).cast('integer'))

In [26]:
from pyspark.sql.functions import *
df_1.select([count(when(col(c).isNull(), c )).alias(c) for c in df_1.columns]).show()

+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|age|workclass|fnlwgt|education|numbers|marital|occupation|relation|race|gender|gain|loss|hourlypay|country|income|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|  1|        0|     1|        0|      1|      0|         0|       1|   1|     1|   2|   2|        2|      1|     1|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+



In [27]:
df.select('workclass').distinct().show()

+----------------+
|       workclass|
+----------------+
|Self-emp-not-inc|
|       workclass|
|       Local-gov|
|       State-gov|
|         Private|
|     Without-pay|
|     Federal-gov|
|    Never-worked|
|               ?|
|    Self-emp-inc|
+----------------+



In [28]:
df = df_1.replace("?",None)

In [29]:
df.select([count(when(col(c).isNull(), c )).alias(c) for c in df.columns]).show()

+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|age|workclass|fnlwgt|education|numbers|marital|occupation|relation|race|gender|gain|loss|hourlypay|country|income|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|  1|      533|     1|        0|      1|      0|       534|       1|   1|     1|   2|   2|        2|    187|     1|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+



In [32]:
df.groupby('country').count().show()

+------------------+-----+
|           country|count|
+------------------+-----+
|       Philippines|   45|
|           Germany|   44|
|          Cambodia|    9|
|            France|    6|
|            Greece|   15|
|            Taiwan|   17|
|              NULL|  187|
|           Ecuador|    6|
|         Nicaragua|    9|
|              Hong|    2|
|              Peru|    4|
|             China|   27|
|             India|   33|
|Holand-Netherlands|    1|
|             Italy|   20|
|              Cuba|   28|
|             South|   22|
|              Iran|   17|
|           Ireland|    8|
|              Laos|    4|
+------------------+-----+
only showing top 20 rows



In [33]:
df = df.fillna(" United-States", subset = ['country'])

In [35]:
df.groupby("occupation").count().show()

+-----------------+-----+
|       occupation|count|
+-----------------+-----+
|            Sales| 1131|
|  Exec-managerial| 1435|
|   Prof-specialty| 1442|
|Handlers-cleaners|  314|
|             NULL|  534|
|    Prof-specialt|    1|
|  Farming-fishing|  257|
|     Craft-repair| 1166|
| Transport-moving|  454|
|  Priv-house-serv|   32|
|  Protective-serv|  182|
|    Other-service|  745|
|     Tech-support|  276|
|Machine-op-inspct|  546|
|     Armed-Forces|    3|
|     Adm-clerical|  993|
|       occupation|    1|
+-----------------+-----+



In [36]:
df = df.fillna(" Prof-specialty", subset = ['occupation'])

In [37]:
df.groupby('workclass').count().show()

+----------------+-----+
|       workclass|count|
+----------------+-----+
|Self-emp-not-inc|  814|
|       workclass|    1|
|            NULL|  533|
|       Local-gov|  656|
|       State-gov|  374|
|         Private| 6395|
|     Without-pay|    3|
|     Federal-gov|  295|
|    Never-worked|    1|
|    Self-emp-inc|  440|
+----------------+-----+



In [40]:
df = df.fillna(" Private", subset = ['workclass'])

In [41]:
df.select([count(when(col(c).isNull(), c )).alias(c) for c in df.columns]).show()

+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|age|workclass|fnlwgt|education|numbers|marital|occupation|relation|race|gender|gain|loss|hourlypay|country|income|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+
|  1|        0|     1|        0|      1|      0|         0|       1|   1|     1|   2|   2|        2|      0|     1|
+---+---------+------+---------+-------+-------+----------+--------+----+------+----+----+---------+-------+------+



In [52]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler , StringIndexer
from pyspark.ml import Pipeline

In [43]:
categorical_cols = [ 'workclass',
 'education',
 'marital',
 'occupation',
 'relation',
 'race',
 'gender',
 'country']
numeric_cols = [ 'age' ,'fnlwgt' , 'numbers', 'gain', 'loss', 'hourlypay']
label = 'income'


In [45]:
indexer = [StringIndexer(inputCol = c , outputCol = f"{c}_index",handleInvalid='keep')for c in categorical_cols]

In [47]:
label_indexer = StringIndexer(inputCol='income', outputCol='label' , handleInvalid='keep')

In [67]:
assembler = VectorAssembler(inputCols = [f'{c}_index' for c in categorical_cols] + numeric_cols, outputCol = 'features',handleInvalid='skip')

In [68]:
lregression = LogisticRegression(featuresCol = 'features', labelCol = 'label')


In [69]:
pipeline = Pipeline(stages = indexer + [ assembler ,label_indexer ,  lregression])

In [70]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)


In [71]:
model = pipeline.fit(train_data)

In [72]:
prediction = model.transform(test_data)

In [75]:
prediction.show(5)

+---+---------+------+---------+-------+-------------+---------------+---------+-----+------+----+----+---------+-------------+------+---------------+---------------+-------------+----------------+--------------+----------+------------+-------------+--------------------+-----+--------------------+--------------------+----------+
|age|workclass|fnlwgt|education|numbers|      marital|     occupation| relation| race|gender|gain|loss|hourlypay|      country|income|workclass_index|education_index|marital_index|occupation_index|relation_index|race_index|gender_index|country_index|            features|label|       rawPrediction|         probability|prediction|
+---+---------+------+---------+-------+-------------+---------------+---------+-----+------+----+----+---------+-------------+------+---------------+---------------+-------------+----------------+--------------+----------+------------+-------------+--------------------+-----+--------------------+--------------------+----------+
| 17|  

In [73]:
prediction.select('label' , 'prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [74]:
prediction.groupby('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  490|
|  0.0|       1.0|  123|
|  1.0|       0.0|  192|
|  0.0|       0.0| 1031|
+-----+----------+-----+



In [77]:
prediction.select('label' , 'prediction').distinct().show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
+-----+----------+



In [78]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [79]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [80]:
evaluator.evaluate(prediction)

0.8284313725490197